## (भिजुअल) च्याट कम्प्लिशन इनफरेन्स अनलाइन एन्डपोइन्ट प्रयोग गर्दै

यो उदाहरणले `Phi-3-vision-128k-instruct` लाई अनलाइन एन्डपोइन्टमा इनफरेन्सको लागि कसरी डिप्लोय गर्ने भनेर देखाउँछ।

### रूपरेखा
* पूर्व-आवश्यकता सेट अप गर्नुहोस्
* डिप्लोय गर्न मोडेल चयन गर्नुहोस्
* इनफरेन्सको लागि डेटा डाउनलोड र तयार गर्नुहोस्
* वास्तविक समय इनफरेन्सको लागि मोडेल डिप्लोय गर्नुहोस्
* एन्डपोइन्ट परीक्षण गर्नुहोस्
* Azure OpenAI शैलीको पेलोड प्रयोग गर्दै एन्डपोइन्ट परीक्षण गर्नुहोस्
* स्रोतहरू सफा गर्नुहोस्


### १. पूर्व-आवश्यकताहरू सेट गर्नुहोस्
* निर्भरता स्थापना गर्नुहोस्  
* AzureML Workspace मा जडान गर्नुहोस्। [SDK प्रमाणीकरण सेट अप गर्नुहोस्](https://learn.microsoft.com/azure/machine-learning/how-to-setup-authentication?tabs=sdk) मा थप जान्नुहोस्। तल `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` र `<SUBSCRIPTION_ID>` प्रतिस्थापन गर्नुहोस्।  
* `azureml` प्रणाली रजिष्ट्रीसँग जडान गर्नुहोस्  


In [ ]:
# Import necessary modules
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    # Try to get the default Azure credential
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # If default credential is not available, use interactive browser credential
    credential = InteractiveBrowserCredential()

try:
    # Try to create an MLClient using the provided credential
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # If MLClient creation fails, enter the details of your AML workspace manually
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<WORKSPACE_NAME>"

# Create an MLClient instance with the provided credentials and workspace details
workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# The models, fine tuning pipelines, and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(credential, registry_name="azureml")

### 2. मोडेललाई अनलाइन अन्त बिन्दुमा तैनाथ गर्नुहोस्  
अनलाइन अन्त बिन्दुहरूले एक स्थायी REST API प्रदान गर्छन् जसलाई मोडेल प्रयोग गर्न आवश्यक पर्ने एप्लिकेसनहरूसँग एकीकृत गर्न प्रयोग गर्न सकिन्छ।  


In [ ]:
# This code checks if the model with the specified name exists in the registry.
# If the model exists, it retrieves the first version of the model and prints its details.
# If the model does not exist, it prints a message indicating that the model was not found.

# model_name: Name of the model to check in the registry
model_name = "Phi-3-vision-128k-instruct"

# Get the list of versions for the specified model name
version_list = list(registry_ml_client.models.list(model_name))

# Check if any versions of the model exist in the registry
if len(version_list) == 0:
    print("Model not found in registry")
else:
    # Get the first version of the model
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    
    # Print the details of the model
    print(
        "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
            foundation_model.name, foundation_model.version, foundation_model.id
        )
    )

In [ ]:
# Import necessary modules
import time
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = model_name[:13] + str(timestamp)
print(f"Creating online endpoint with name: {online_endpoint_name}")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description=f"Online endpoint for {foundation_model.name}, for visual chat-completion task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# This code creates a deployment for the online endpoint.
# It sets the deployment name, endpoint name, model, instance type, instance count, and request settings.
# It also sets the liveness probe and readiness probe settings.
# Finally, it updates the traffic distribution for the endpoint.

"""
from azure.ai.ml.entities import OnlineRequestSettings, ProbeSettings

# create a deployment
deployment_name = "phi-3-vision"
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_NC48ads_A100_v4",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=180000,
        max_queue_wait_ms=500,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=49,
        success_threshold=1,
        timeout=299,
        period=180,
        initial_delay=180,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=10,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()
"""

### 3. नमूना डाटा प्रयोग गरेर अन्तिम बिन्दु परीक्षण गर्नुहोस्

हामी तल सिर्जना गरिएको json प्रयोग गरेर मोडेललाई नमूना अनुरोध पठाउनेछौं।


In [ ]:
# Import necessary modules
import json
import os

# Define the test JSON payload
test_json = {
    "input_data": {
        "input_string": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                        },
                    },
                    {
                        "type": "text",
                        "text": "What is shown in this image? Be extremely detailed and specific.",
                    },
                ],
            },
        ],
        "parameters": {"temperature": 0.7, "max_new_tokens": 2048},
    }
}

# Save the JSON object to a file
sample_score_file_path = os.path.join(".", "sample_chat_completions_score.json")
with open(sample_score_file_path, "w") as f:
    json.dump(test_json, f, indent=4)

# Print the input payload
print("Input payload:\n")
print(test_json)

In [ ]:
# Import necessary modules
import pandas as pd

# score the sample_chat_completions_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file=sample_score_file_path,
)
print("Raw JSON Response: \n", response, "\n")

# Parse the JSON string
json_data = json.loads(response)

# Convert the parsed JSON to a DataFrame
response_df = pd.DataFrame([json_data])
print("Generated Text:\n", response_df["output"].iloc[0])

### 4. Azure OpenAI शैलीको पेलोड प्रयोग गरेर अन्तिम बिन्दु परीक्षण गर्नुहोस्

हामी मोडेलमा Azure OpenAI शैलीको पेलोडसहित एउटा नमूना अनुरोध पठाउनेछौं।


In [ ]:
# This code defines a JSON payload for testing the online endpoint with Azure OpenAI style payload.
# It includes the model name, a list of messages with user role and content (image URL and text),
# temperature, and max_new_tokens.

aoai_test_json = {
    "model": foundation_model.name,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                    },
                },
                {
                    "type": "text",
                    "text": "What is shown in this image? Be extremely detailed and specific.",
                },
            ],
        }
    ],
    "temperature": 0.7,
    "max_new_tokens": 2048,
}

In [ ]:
# Get the scoring uri
scoring_uri = workspace_ml_client.online_endpoints.get(
    name=online_endpoint_name
).scoring_uri
# Update the scoring uri to use for AOAI
aoai_format_scoring_uri = scoring_uri.replace("/score", "/v1/chat/completions")

# Get the key for data plane operation
data_plane_token = workspace_ml_client.online_endpoints.get_keys(
    name=online_endpoint_name
).primary_key

In [ ]:
import urllib.request
import json

# Prepare request
body = str.encode(json.dumps(aoai_test_json))
url = aoai_format_scoring_uri
api_key = data_plane_token

headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + api_key)}
req = urllib.request.Request(url, body, headers)

# Send request & get response
try:
    response = urllib.request.urlopen(req)
    result = response.read().decode("utf-8")
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

### ५. अनलाइन एन्डप्वाइन्ट मेटाउनुहोस्
अनलाइन एन्डप्वाइन्ट मेटाउन नबिर्सनुहोस्, नत्र एन्डप्वाइन्टले प्रयोग गरेको कम्प्युटको लागि बिलिङ मिटर चलिरहनेछ।


In [ ]:
#Delete Workspace
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait(#)


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
